## VAR model for benchmark performance

### import statements

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
from statsmodels.tsa.api import VAR
from statsmodels.tsa.stattools import adfuller

## initialising data (adf test )

In [28]:
training_data=pd.read_csv('../data/final/training_model_data.csv',header=0)
test_data=pd.read_csv('../data/final/incompl_test_model_data.csv',header=0)
test_data=test_data[training_data.columns]
combined=pd.concat([training_data,test_data],axis=0)
df_subset=combined[(combined['year']>=2006) & (combined['year']<=2023)]
df_subset['country_pair']=df_subset['country_a']+df_subset['country_b']
df_subset.head()
df_subset=df_subset.drop(columns=['tradeagreementindex','sentiment_index','country_a','country_b'])
df_long = df_subset.melt(
    id_vars=['year','country_pair'],
    value_vars=['bec_1','bec_2','bec_3','bec_4','bec_5','bec_6','bec_7','bec_8'],
    var_name='sector',
    value_name='value'
)
df_long=df_long.sort_values(['country_pair','sector','year'])
grouped=df_long.groupby(['country_pair','sector'])
results_list = []
for (pair, sec), group_data in grouped:
    ts = group_data.set_index('year')['value'].dropna()


    adf_out = adfuller(ts, autolag='AIC')
    adf_statistic  = adf_out[0]
    p_value        = adf_out[1]
    used_lag       = adf_out[2]
    nobs           = adf_out[3]
    critical_values= adf_out[4]
    
    results_list.append({
        'country_pair': pair,
        'sector': sec,
        'adf_stat': adf_statistic,
        'p_value': p_value,
        'used_lag': used_lag,
        'nobs': nobs
    })

results_df = pd.DataFrame(results_list)
print(results_df)


     country_pair sector   adf_stat       p_value  used_lag  nobs
0          AREAUS  bec_1  -3.124212  2.481254e-02         5    12
1          AREAUS  bec_2  -2.817751  5.578257e-02         7    10
2          AREAUS  bec_3   3.498822  1.000000e+00         7    10
3          AREAUS  bec_4 -12.485611  3.023461e-23         7    10
4          AREAUS  bec_5  -1.953536  3.073098e-01         7    10
...           ...    ...        ...           ...       ...   ...
2443       VNMUSA  bec_4   9.862133  1.000000e+00         7    10
2444       VNMUSA  bec_5   1.455103  9.973506e-01         7    10
2445       VNMUSA  bec_6   4.176970  1.000000e+00         7    10
2446       VNMUSA  bec_7   1.192659  9.959348e-01         7    10
2447       VNMUSA  bec_8  -4.548111  1.608114e-04         7    10

[2448 rows x 6 columns]


In [29]:
median_lags=results_df.groupby('country_pair')['used_lag'].median().reset_index()
median_lags.to_csv('../data/final/median_adf_lags.csv',header=True)

In [ ]:
def adf_test(series, title=''):
    """Perform ADF test and print results."""
    print(f'Augmented Dickey-Fuller Test: {title}')
    result = adfuller(series.dropna(), autolag='AIC')
    labels = ['ADF Statistic', 'p-value', '# Lags Used', '# Observations Used']
    out = dict(zip(labels, result[0:4]))
    for key, val in out.items():
        print(f"   {key}: {val}")
    for key, val in result[4].items():
        print(f"   Critical Value {key}: {val}")
    print("")

# Example: check stationarity for each column
for col in df.columns:
    adf_test(df[col], title=col)

# If non-stationary, consider differencing
df_diff = df.diff().dropna()

# Re-check with ADF after differencing
for col in df_diff.columns:
    adf_test(df_diff[col], title=col)

In [30]:
training_data=pd.read_csv('../data/final/training_model_data.csv',header=0)
test_data=pd.read_csv('../data/final/incompl_test_model_data.csv',header=0)
test_data=test_data[training_data.columns]
combined=pd.concat([training_data,test_data],axis=0)

In [31]:
import numpy as np
import pandas as pd
from statsmodels.tsa.stattools import adfuller

def is_stationary(series, significance=0.05):
    """
    Runs Augmented Dickey-Fuller test on a single series.
    Returns True if stationary at given significance level, otherwise False.
    """
    # ADF test
    result = adfuller(series, autolag='AIC')
    p_value = result[1]
    return p_value < significance

def check_stationarity(df, significance=0.05):
    """
    Checks if ALL columns in df are stationary.
    Returns True if all are stationary, else False.
    """
    for col in df.columns:
        # Drop NA to avoid errors
        series = df[col].dropna()
        if len(series) < 10:
            # If there's too little data, skip or treat as non-stationary
            return False
        # If any column is non-stationary, return False immediately
        if not is_stationary(series, significance=significance):
            return False
    # If we get here, all columns are stationary
    return True

def make_stationary(df_in, max_diffs=2, significance=0.05):
    """
    Iteratively check stationarity for all columns.
    If any are non-stationary, difference the entire DataFrame once and re-check.
    Continue until everything is stationary or we reach max_diffs.
    
    Returns:
      (df_out, diff_count)
        df_out     -> final (differenced) DataFrame
        diff_count -> how many times the dataset was differenced
    """
    df_out = df_in.copy()
    diff_count = 0
    
    while diff_count < max_diffs:
        if check_stationarity(df_out, significance=significance):
            # All columns stationary
            break
        else:
            # Difference all columns once
            df_out = df_out.diff().dropna()
            diff_count += 1
    
    # By now, either it's stationary or we hit max_diffs
    return df_out, diff_count

In [33]:
from statsmodels.tsa.api import VAR

def run_var_for_pair(df_pair, date_col='year', significance=0.05, max_diffs=2, max_lags=5, nobs=3):
    """
    1) Splits data into train/test.
    2) Automates stationarity checks & differencing.
    3) Selects the best VAR order by AIC.
    4) Fits VAR and forecasts nobs steps.
    5) Inverts differencing (if needed).
    6) Returns results + model.
    """
    # 1) Set date index & sort
    if date_col in df_pair.columns:
        df_pair = df_pair.set_index(date_col)
    df_pair = df_pair.sort_index()
    
    # 2) Train/test split
    df_train, df_test = df_pair.iloc[:-nobs], df_pair.iloc[-nobs:]
    
    # 3) Make training data stationary with auto-differencing
    df_stationary, diff_count = make_stationary(df_train, max_diffs=max_diffs, significance=significance)
    
    # 4) Select best VAR order with AIC
    model = VAR(df_stationary)
    best_lag = None
    best_aic = np.inf
    for lag in range(1, max_lags+1):
        try:
            result = model.fit(lag)
            if result.aic < best_aic:
                best_aic = result.aic
                best_lag = lag
        except:
            # Some models might fail to converge for certain lags
            pass
    
    if best_lag is None:
        raise ValueError("No suitable VAR lag found. Check data or reduce max_lags.")
    
    # Fit final model
    model_fitted = model.fit(best_lag)
    
    # 5) Forecast nobs steps
    forecast_input = df_stationary.values[-best_lag:]
    fc = model_fitted.forecast(y=forecast_input, steps=nobs)
    df_fc = pd.DataFrame(fc, index=df_test.index, columns=[col + "_diff" for col in df_stationary.columns])
    
    # 6) If we differenced, invert it to original scale
    #    Example for a single differencing pass
    df_inverted = None
    if diff_count > 0:
        df_inverted = df_fc.copy()
        # We do repeated cumulative sums depending on how many times we've differenced
        # Example for 1 difference:
        if diff_count == 1:
            for col in df_stationary.columns:
                df_inverted[col + '_forecast'] = df_train[col].iloc[-1] + df_inverted[col + '_diff'].cumsum()
        elif diff_count == 2:
            for col in df_stationary.columns:
                # First invert the second difference
                df_inverted[col + '_1d'] = (df_train[col].iloc[-1] - df_train[col].iloc[-2]) + df_inverted[col + '_diff'].cumsum()
                # Then invert the first difference
                df_inverted[col + '_forecast'] = df_train[col].iloc[-1] + df_inverted[col + '_1d'].cumsum()
        # etc. If you want to be robust to diff_count > 2, just loop.
        
    # 7) Calculate simple forecast accuracy metrics
    def forecast_accuracy(forecast, actual):
        mape = np.mean(np.abs(forecast - actual) / np.abs(actual))  
        rmse_val = np.sqrt(np.mean((forecast - actual)**2))
        return {"mape": mape, "rmse": rmse_val}
    
    metrics = {}
    # Evaluate either df_fc or df_inverted depending on your scale of interest
    df_evaluation = df_inverted if diff_count > 0 else df_fc
    
    for col in df_stationary.columns:
        fc_col = col + ('_forecast' if diff_count > 0 else '_diff')
        if fc_col not in df_evaluation.columns:
            continue
        y_true = df_test[col]
        y_pred = df_evaluation[fc_col]
        # Filter out any NA
        y_true, y_pred = y_true.align(y_pred, join='inner')
        if len(y_true) > 0:
            metrics[col] = forecast_accuracy(y_pred, y_true)
    
    # Return final results
    return {
        'diff_count': diff_count,
        'best_lag': best_lag,
        'model_fitted': model_fitted,
        'df_fc_diff': df_fc,       # forecast in differenced scale
        'df_fc_inverted': df_inverted,  # forecast in original scale if diff_count>0
        'metrics': metrics
    }

In [ ]:
results_all_pairs = {}
grouped = combined.groupby(['country_a', 'country_b'])
for (exp, imp), df_pair in grouped:
    result = run_var_for_pair(
        df_pair,
        date_col='date',
        significance=0.05,  # or 0.01, your choice
        max_diffs=2,        # differ up to twice
        max_lags=12,
        nobs=4
    )
    pair_key = f"{exp}_{imp}"
    results_all_pairs[pair_key] = result

     country_a country_b         bec_1         bec_2         bec_3  \
3570       ARE       AUS  0.000000e+00  0.000000e+00  0.000000e+00   
3571       ARE       AUS  1.764716e+08  5.812547e+08  1.131179e+08   
3572       ARE       AUS  2.186282e+08  5.242286e+08  1.421974e+08   
3573       ARE       AUS  0.000000e+00  0.000000e+00  0.000000e+00   
3574       ARE       AUS  0.000000e+00  0.000000e+00  0.000000e+00   

             bec_4         bec_5         bec_6         bec_7         bec_8  \
3570  0.000000e+00  0.000000e+00  0.000000e+00  0.000000e+00  1.060739e+09   
3571  6.507964e+08  3.311616e+08  1.274907e+07  2.253624e+07  7.838146e+07   
3572  7.929628e+08  3.152631e+08  1.360508e+07  2.231833e+07  7.764083e+07   
3573  0.000000e+00  0.000000e+00  0.000000e+00  0.000000e+00  1.814150e+09   
3574  0.000000e+00  0.000000e+00  0.000000e+00  0.000000e+00  2.118919e+09   

      sentiment_index  tradeagreementindex  year  
3570         0.548505                    0  2006  
3571    

In [ ]:
# from sklearn.metrics import mean_absolute_error, mean_squared_error

# # Align true values and forecasts
# actual = df.loc[test_data.index, :]  # the actual values in the original scale
# preds = final_forecast.loc[test_data.index, :]

# mae_export = mean_absolute_error(actual["Export"], preds["Export"])
# mae_import = mean_absolute_error(actual["Import"], preds["Import"])

# mse_export = mean_squared_error(actual["Export"], preds["Export"])
# mse_import = mean_squared_error(actual["Import"], preds["Import"])

# print("MAE (Export):", mae_export)
# print("MAE (Import):", mae_import)
# print("MSE (Export):", mse_export)
# print("MSE (Import):", mse_import)